In [1]:
from dask.distributed import LocalCluster, Client, progress
cluster = LocalCluster(ip="0.0.0.0", threads_per_worker=2, n_workers=16)
client = Client(cluster)  # memory_limit='64GB'
cluster

2023-05-10 17:20:18,723 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-y_f2ln19', purging
2023-05-10 17:20:18,724 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-cgqgm9q1', purging
2023-05-10 17:20:18,724 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-wgipvta8', purging
2023-05-10 17:20:18,724 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-w5da2ppb', purging
2023-05-10 17:20:18,724 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-0npiprh6', purging
2023-05-10 17:20:18,725 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-z0f76aac', purging
2023-05-10 17:20:18,725 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-un5lqrh1', purging

LocalCluster(5f5fcd83, 'tcp://136.172.124.5:41985', workers=16, threads=32, memory=503.45 GiB)

In [2]:
from aqua import Reader, catalogue
#catalogue();

In [10]:
model_list = ["IFS"]
exp_name = ["tco3999-ng5","tco2559-ng5","tco1279-orca025"]
reader = Reader(model='IFS', exp = 'tco2559-ng5', source="ICMGG_atm2d", freq='monthly', regrid='r100')
data = reader.retrieve()

Task exception was never retrieved
future: <Task finished name='Task-89394' coro=<Client._gather.<locals>.wait() done, defined at /work/bb1153/b382216/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/client.py:2187> exception=AllExit()>
Traceback (most recent call last):
  File "/work/bb1153/b382216/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/client.py", line 2196, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-89395' coro=<Client._gather.<locals>.wait() done, defined at /work/bb1153/b382216/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/client.py:2187> exception=AllExit()>
Traceback (most recent call last):
  File "/work/bb1153/b382216/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/client.py", line 2196, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-89396' coro=<Client._ga

In [11]:
temp_day = reader.timmean(data['2t'])
global_mean_temp=reader.fldmean(temp_day)
global_mean_temp.plot()


AttributeError: 'tuple' object has no attribute 'tb_frame'